# **Product Report**

## **Why?**

### **This report consolidates key product metrics and behaviors.**

### 

1. Gathers essential fields such as product name, category, subcategroy, and cost.
2. Segments products revenue to identify High-Performers, Mid-Range, or Low-Performers.
3. Aggregates product-level metrics:

    - total orders
    - total sales
    - total quantity sold
    - total customers (unique)
    - lifespan (in months)

### 

4. Calculates valuable KPIs:

    - recency (months since last sale)
    - average order revenue (AOR)
    - average monthly revenue

Create Report: gold.report_products

In [2]:
IF OBJECT_ID('gold.report_products', 'V') IS NOT NULL
    DROP VIEW gold.report_products;

Commands completed successfully.

Total execution time: 00:00:00.011

In [3]:
CREATE VIEW gold.report_products AS

WITH base_query AS (
    -- Base Query: Retrieves core columns from fact_sales and dim_products
    SELECT
        f.order_number, 
        f.order_date,
        f.customer_key,
        f.sales_amount,
        f.quantity, 
        p.product_key,
        p.product_name,
        p.category,
        p.subcategory,
        p.cost
    FROM gold.fact_sales f
    LEFT JOIN gold.dim_products p 
        ON f.product_key = p.product_key
    WHERE order_date IS NOT NULL -- only consider vlaid sales dates
),

product_aggregations AS (
-- Product Aggregations: Summarizes key metrics at the product level
SELECT
    product_key,
    product_name,
    category,
    subcategory, 
    cost, 
    DATEDIFF(MONTH, MIN(order_date), MAX(order_date)) AS lifespan, 
    MAX(order_date) AS last_sale_date,
    COUNT(DISTINCT order_number) AS total_orders,
    COUNT(DISTINCT customer_key) AS total_customers, 
    SUM(sales_amount) AS total_sales, 
    SUM(quantity) AS total_quantity,
    ROUND(AVG(CAST(sales_amount AS FLOAT) / NULLIF(quantity, 0)), 1) AS avg_selling_price
FROM base_query

GROUP BY
    product_key, 
    product_name, 
    category,
    subcategory,
    cost
)

-- Final Query: Combines all product results into one output

SELECT
    product_key,
    product_name, 
    category,
    subcategory,
    cost, 
    last_sale_date,
    DATEDIFF(MONTH, last_sale_date, GETDATE()) AS recency_in_months,
    CASE
        WHEN total_sales > 50000 THEN 'High-Performer'
        WHEN total_sales >= 10000 THEN 'Mid-Range'
        ELSE 'Low-Performer'
    END AS product_segment,
    lifespan,
    total_orders,
    total_sales,
    total_quantity,
    total_customers,
    avg_selling_price,
    -- Average Order Revenue (AOR)
    CASE
        WHEN total_orders = 0 THEN 0
        ELSE total_sales / total_orders
    END AS avg_order_revenue,

    -- Average Monthly Revenue
    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_revenue

FROM product_aggregations

Commands completed successfully.

Total execution time: 00:00:00.027

In [4]:
SELECT TOP (20) [customer_key]
      ,[customer_number]
      ,[customer_name]
      ,[age]
      ,[age_group]
      ,[customer_segment]
      ,[last_order_date]
      ,[recency]
      ,[total_orders]
      ,[total_sales]
      ,[total_quantity]
      ,[total_products]
      ,[lifespan]
      ,[avg_order_value]
      ,[avg_monthly_spend]
  FROM [DataWarehouse].[Gold].[report_customers]

(20 rows affected)

Total execution time: 00:00:01.006

customer_key,customer_number,customer_name,age,age_group,customer_segment,last_order_date,recency,total_orders,total_sales,total_quantity,total_products,lifespan,avg_order_value,avg_monthly_spend
1,AW00011000,Jon Yang,54,50 and above,VIP,2013-05-03,148,3,8249,8,8,28,2749,294
2,AW00011001,Eugene Huang,49,40-49,VIP,2013-12-10,141,3,6384,11,10,35,2128,182
3,AW00011002,Ruben Torres,54,50 and above,VIP,2013-02-23,151,3,8114,4,4,25,2704,324
4,AW00011003,Christy Zhu,52,50 and above,VIP,2013-05-10,148,3,8139,9,9,29,2713,280
5,AW00011004,Elizabeth Johnson,46,40-49,VIP,2013-05-01,148,3,8196,6,6,28,2732,292
6,AW00011005,Julio Ruiz,49,40-49,VIP,2013-05-02,148,3,8121,6,6,29,2707,280
7,AW00011006,Janet Alvarez,49,40-49,VIP,2013-05-14,148,3,8119,5,5,28,2706,289
8,AW00011007,Marco Mehta,56,50 and above,VIP,2013-03-19,150,3,8211,8,8,26,2737,315
9,AW00011008,Rob Verhoff,50,50 and above,VIP,2013-03-02,150,3,8106,7,7,26,2702,311
10,AW00011009,Shannon Carlson,56,50 and above,VIP,2013-05-09,148,3,8091,5,5,28,2697,288
